In [2]:
import pandas as pd 
import numpy as np

In [85]:
df = pd.read_excel("30 mutal funds.xlsx", index_col=0, parse_dates=True)
df

,PGIM India Small Cap Fund Direc,SBI Magnum Taxgain Scheme Regular Dividend Payout,Baroda Multi Cap Fund - Plan A - Dividend Payout,Baroda Banking and Financial Services Fund Plan B (Direct) Dividend Payout Option,ICICI Pru Life-Secure Opportuni,Reliance Health Energy Fund 1,Franklin India Equity Hybrid Fund Dividend Payout,Reliance Pension Pure Equity Fund 2,Templeton India Equity Income Fund Dividend Payout,Mirae Asset Arbitrage Fund Regu,...,Canara Robeco Overnight Fund Direct Growth,Mirae Asset Focused Fund Regular Dividend Payout,L&amp;T Tax Advantage Fund Dividend Payout,Nippon India Asset Allocator Fo,Motilal Oswal Long Term Equity Fund - Direct Plan - Growth,SBI Retirement Benefit Fund - A,DSP Top 100 Equity Fund Direct Plan Dividend Reinvestment,PGIM India Insta Cash Fund Monthly Dividend Reinvestment,ITI Value Fund Regular Growth,DSP Nifty Next 50 Index Fund Regular Dividend Reinvestment
date,,,,,,,,,,,,,,,,,,,,,
2023-02-16,10.87,53.844601,41.053299,20.009800,13.2879,31.068501,24.729500,35.231098,20.400400,11.051,...,1151.518188,18.572001,24.055901,13.1463,30.456800,13.8074,23.910999,111.140404,10.3426,14.2115
2023-02-17,10.90,53.573502,40.879398,19.792700,13.2844,31.048800,24.650400,35.181499,20.368700,11.058,...,1151.714355,18.490999,23.916401,13.1013,30.337400,13.7909,23.760000,111.158997,10.2915,14.1231
2023-02-20,10.90,53.348099,40.689800,19.636499,13.2883,30.980600,24.623100,35.128399,20.341400,11.057,...,1152.305542,18.492001,23.814600,13.1124,30.286301,13.7623,23.695000,111.213303,10.2636,14.0787
2023-02-21,10.84,53.307499,40.576599,19.619699,13.2873,31.106199,24.630100,35.092701,20.316401,11.062,...,1152.508667,18.511000,23.795000,13.0932,30.202299,13.7618,23.634001,111.234200,10.2503,14.0526
2023-02-22,10.70,52.798801,40.171902,19.280701,13.2874,30.856001,24.400400,34.617802,20.153400,11.069,...,1152.709351,18.292000,23.505501,12.9938,29.817600,13.6382,23.407000,111.252701,10.1243,13.8544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-15,10.53,50.480701,41.396099,19.206200,13.5232,33.320099,25.289400,35.771599,20.087099,11.232,...,1169.505859,18.926001,22.750099,13.8434,31.878799,14.2540,22.701000,112.194504,10.6829,15.0609
2023-05-16,10.54,50.445301,41.344200,19.187401,13.5277,33.386200,25.249500,35.623100,20.105499,11.238,...,1169.717896,18.907000,22.777500,13.8321,31.928400,14.2866,22.618999,112.218399,10.6609,15.0819
2023-05-17,10.59,50.373699,41.308800,19.119101,13.5287,33.255798,25.184700,35.449699,20.038700,11.237,...,1169.920044,18.850000,22.759399,13.8057,31.931801,14.2655,22.535000,112.244499,10.6742,14.9972


In [7]:
df_return = (df.pct_change()).dropna(axis=0)
df_return.to_clipboard(excel=True)

In [15]:
df_return.loc[:, (df_return.mean() < 0)].mean() * 100

PGIM India Small Cap Fund Direc                                                     -0.040456
SBI Magnum Taxgain Scheme Regular Dividend Payout                                   -0.100634
Baroda Banking and Financial Services Fund Plan B (Direct) Dividend Payout Option   -0.061177
Templeton India Equity Income Fund Dividend Payout                                  -0.033753
L&amp;T Tax Advantage Fund Dividend Payout                                          -0.088937
DSP Top 100 Equity Fund Direct Plan Dividend Reinvestment                           -0.093226
dtype: float64

### Beta Estimation

#### market index data collector

In [31]:
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()
sp500 = pdr.get_data_yahoo("^GSPC", start = "2023-02-16", end = "2023-05-22")

[*********************100%***********************]  1 of 1 completed


In [87]:
df.merge(sp500, left_index=True, right_index=True, how='left').apply(lambda x: x.fillna(x.mean())).to_clipboard(excel=True)

In [32]:
sp500

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-02-16,4114.750000,4136.540039,4089.489990,4090.409912,4090.409912,4143660000
2023-02-17,4077.389893,4081.510010,4047.949951,4079.090088,4079.090088,4045480000
2023-02-21,4052.350098,4052.350098,3995.189941,3997.340088,3997.340088,4121590000
2023-02-22,4001.830078,4017.370117,3976.899902,3991.050049,3991.050049,4079320000
2023-02-23,4018.600098,4028.300049,3969.189941,4012.320068,4012.320068,3952940000
...,...,...,...,...,...,...
2023-05-15,4126.649902,4141.250000,4110.270020,4136.279785,4136.279785,3562170000
2023-05-16,4127.950195,4135.540039,4109.859863,4109.899902,4109.899902,3654200000
2023-05-17,4122.850098,4164.669922,4113.620117,4158.770020,4158.770020,4039080000


In [38]:
market_return = sp500.pct_change().dropna(axis=0)['Adj Close']

In [47]:
combine_df = df_return.merge(market_return, left_index=True, right_index=True, how='left')

In [52]:
combine_df['Adj Close'] = combine_df['Adj Close'].fillna(combine_df['Adj Close'].mean())

In [64]:
betas = []
for col in df_return.columns:
    beta, intercept = np.polyfit(combine_df.iloc[:, -1].values, combine_df[col].values , deg=1)
    betas.append(beta)
betas = {name: beta for name, beta in zip(combine_df.columns[:-1], betas)}
betas
# combine_df

{'PGIM India Small Cap Fund Direc': 0.10768010752341564,
 'SBI Magnum Taxgain Scheme Regular Dividend Payout': -0.005171769287415163,
 'Baroda Multi Cap Fund - Plan A - Dividend Payout': 0.0878623285842141,
 'Baroda Banking and Financial Services Fund Plan B (Direct) Dividend Payout Option': 0.17309580658810583,
 'ICICI Pru Life-Secure Opportuni': 0.00373956943221765,
 'Reliance Health Energy Fund 1': -0.02393341145495783,
 'Franklin India Equity Hybrid Fund Dividend Payout': 0.05581913803079092,
 'Reliance Pension Pure Equity Fund 2': 0.08551682479070705,
 'Templeton India Equity Income Fund Dividend Payout': 0.02610136437544716,
 'Mirae Asset Arbitrage Fund Regu': -0.011640381837823986,
 'Canara Robeco Small Cap Reg DP': 0.05054240584639934,
 'ICICI Pru Life-Maximiser Fund V': 0.08791960591653955,
 'Quantum India ESG Equity Fund Regular Growth': 0.06900532415932545,
 'Union Midcap Fund Regular Reinv': 0.06951880097372268,
 'Invesco India Growth Opportunities Fund Dividend Reinvestmen

In [68]:
beta_df = pd.DataFrame(betas, index=["Beta"]).T
beta_df.to_clipboard(excel=True)

In [69]:
combine_df['Adj Close'].to_clipboard(excel=True)

In [3]:
risk_free = pd.read_csv(r"C:\Users\USER\Python Programme\uni\portfolio_management\Assigment 2\daily-treasury-rates.csv", index_col=0, parse_dates=True)
risk_free

,4 WEEKS BANK DISCOUNT,4 WEEKS COUPON EQUIVALENT,8 WEEKS BANK DISCOUNT,8 WEEKS COUPON EQUIVALENT,13 WEEKS BANK DISCOUNT,13 WEEKS COUPON EQUIVALENT,17 WEEKS BANK DISCOUNT,17 WEEKS COUPON EQUIVALENT,26 WEEKS BANK DISCOUNT,26 WEEKS COUPON EQUIVALENT,52 WEEKS BANK DISCOUNT,52 WEEKS COUPON EQUIVALENT
Date,,,,,,,,,,,,
2023-05-19,5.41,5.52,5.11,5.24,5.12,5.27,5.20,5.38,5.11,5.33,4.77,5.03
2023-05-18,5.38,5.49,5.06,5.19,5.14,5.29,5.18,5.36,5.13,5.35,4.76,5.02
2023-05-17,5.39,5.50,4.87,4.99,5.10,5.25,5.11,5.28,5.05,5.27,4.66,4.91
2023-05-16,5.45,5.56,4.80,4.92,5.07,5.22,4.99,5.16,5.02,5.24,4.63,4.88
2023-05-15,5.49,5.61,4.80,4.92,5.06,5.21,5.01,5.18,4.99,5.20,4.56,4.79
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-09,4.17,4.24,4.44,4.53,4.55,4.67,4.60,4.74,4.69,4.87,4.48,4.70
2023-01-06,4.14,4.21,4.43,4.52,4.51,4.62,4.61,4.75,4.65,4.83,4.50,4.72
2023-01-05,4.12,4.19,4.44,4.53,4.51,4.62,4.62,4.76,4.68,4.86,4.56,4.79


In [8]:
risk_free.iloc[:, -2] / 100 / (52 * 7)

Date
2023-05-19    0.000131
2023-05-18    0.000131
2023-05-17    0.000128
2023-05-16    0.000127
2023-05-15    0.000125
                ...   
2023-01-09    0.000123
2023-01-06    0.000124
2023-01-05    0.000125
2023-01-04    0.000123
2023-01-03    0.000124
Name: 52 WEEKS BANK DISCOUNT, Length: 97, dtype: float64

In [4]:
combine_df_risk_free = combine_df.merge(risk_free.iloc[:, -2] / 100 / (52 * 7), left_index=True, right_index=True, how="left").apply(lambda x: x.fillna(x.mean()))
combine_df_risk_free

NameError: name 'combine_df' is not defined

In [84]:
combine_df_risk_free.to_clipboard(excel=True)